In [1]:
import pandas as pd
java = pd.read_csv('./java/java_mb_KG.txt',names=['user_id','item_id','behavior','timestamp'],sep='\t')
java

,user_id,item_id,behavior,timestamp
0,1,305,watch,1406863244
1,1,172,done,1410874261
2,1,54,done,1410874265
3,1,54,duration,1410874275
4,1,314,done,1410913417
...,...,...,...,...
2489,183,294,done,1429083169
2490,183,294,tag,1429083255
2491,183,294,duration,1429083393
2492,183,357,done,1429084813


In [3]:
k=1

In [4]:
user_list = list(set(java['user_id'].tolist()))

In [7]:
for u in user_list:
    if u==1:
        java_test = java[java['user_id']==1][-k:]
    else:
        java_test = pd.concat([java_test, java[java['user_id']==u][-k:]])
java_test

,user_id,item_id,behavior,timestamp
75,1,1,tag,1429618603
79,2,1,watch,1429684938
99,3,35,duration,1419596727
103,4,1,watch,1426846556
109,5,1,watch,1427458885
...,...,...,...,...
2470,179,301,duration,1427968846
2475,180,1,tag,1428140566
2479,181,1,watch,1420420045
2483,182,1,watch,1411123740


In [8]:
java.groupby('user_id').count()

,item_id,behavior,timestamp
user_id,,,
1,76,76,76
2,4,4,4
3,20,20,20
4,4,4,4
5,6,6,6
...,...,...,...
179,16,16,16
180,5,5,5
181,4,4,4


In [11]:
java_pro = java.groupby('user_id').count().reset_index()
java_pro[java_pro['item_id']>=k+3]

,user_id,item_id,behavior,timestamp
0,1,76,76,76
1,2,4,4,4
2,3,20,20,20
3,4,4,4,4
4,5,6,6,6
...,...,...,...,...
178,179,16,16,16
179,180,5,5,5
180,181,4,4,4
181,182,4,4,4


In [12]:
java[~java.index.isin(java_test.index)]

,user_id,item_id,behavior,timestamp
0,1,305,watch,1406863244
1,1,172,done,1410874261
2,1,54,done,1410874265
3,1,54,duration,1410874275
4,1,314,done,1410913417
...,...,...,...,...
2488,183,1,tag,1429083139
2489,183,294,done,1429083169
2490,183,294,tag,1429083255
2491,183,294,duration,1429083393


In [16]:
for k in range(1,11):
    java_pro = java.groupby('user_id').count().reset_index()
    user_list = list(set(java_pro[java_pro['item_id']>=k+3]['user_id'].tolist()))
    for u in user_list:
        if u==1:
            java_test = java_pro[java_pro['user_id']==1][-k:]
        else:
            java_test = pd.concat([java_test, java_pro[java_pro['user_id']==u][-k:]])
    java_train = java[~java.index.isin(java_test.index)]
    testpath = './java/java_mb_KG_test'+str(k)+'.txt'
    java_test = java[java['user_id'].isin(user_list)]
    java_test.to_csv(testpath, sep='\t', index=False, header=False)
    trainpath = './java/java_mb_KG_train'+str(k)+'.txt'
    java_train.to_csv(trainpath, sep='\t', index=False, header=False)


In [29]:
java_inter = java.groupby('user_id').agg('item_id').unique().reset_index()
java_inter['inter_len'] = java_inter['item_id'].apply(lambda x: len(x))
java_inter

,user_id,item_id,inter_len
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",33
1,2,"[363, 1]",2
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",11
3,4,"[363, 1]",2
4,5,"[1, 294, 363]",3
...,...,...,...
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]",8
179,180,[1],1
180,181,"[363, 1]",2
181,182,"[294, 54, 1]",3


In [30]:
java_inter[java_inter['inter_len']>3]

,user_id,item_id,inter_len
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",33
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",11
5,6,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",33
7,8,"[294, 295, 35, 297, 298, 54, 300, 301, 302, 30...",17
8,9,"[327, 321, 294, 301, 300, 310, 173, 336, 350, ...",12
...,...,...,...
164,165,"[1, 294, 295, 35, 297, 298, 300]",7
175,176,"[363, 1, 294, 300, 310, 321, 322, 173, 342, 361]",10
177,178,"[298, 311, 1, 331]",4
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]",8


In [49]:
java_new = pd.DataFrame(columns=['user_id','item_id_list','seq_len','target','inter_list','position'])

In [43]:
java_inter[java_inter['user_id']==1]['inter_len'].tolist()[0]

33

In [58]:
java_inter[java_inter['user_id']==1]['item_id'][0].tolist()

[305,
 172,
 54,
 314,
 1,
 350,
 352,
 301,
 302,
 310,
 312,
 337,
 319,
 311,
 336,
 210,
 341,
 343,
 321,
 323,
 300,
 294,
 295,
 35,
 297,
 316,
 346,
 218,
 223,
 306,
 173,
 331,
 322]

In [85]:
alldata = []
for user in java_inter['user_id'].tolist():
    seq_len = java_inter[java_inter['user_id']==user]['inter_len'].tolist()[0]
    total_seq = java_inter[java_inter['user_id']==1]['item_id'][0].tolist()
    for i in range(1,seq_len+1):
        for j in range(1,seq_len+1-i):
            data = []
            inter_list = total_seq[:i]
            position = i
            item_id_list = total_seq[:i+j]
            target = item_id_list[-1]
            seq_len = i+j
            data.append(user)
            data.append(item_id_list)
            data.append(seq_len)
            data.append(target)
            data.append(inter_list)
            data.append(position)
            alldata.append(data)
java_new = pd.DataFrame(alldata,columns = ['user_id','item_id_list','seq_len','target','inter_list','position'])
java_new

,user_id,item_id_list,seq_len,target,inter_list,position
0,1,"[305, 172]",2,172,[305],1
1,1,"[305, 172, 54]",3,54,[305],1
2,1,"[305, 172, 54, 314]",4,314,[305],1
3,1,"[305, 172, 54, 314, 1]",5,1,[305],1
4,1,"[305, 172, 54, 314, 1, 350]",6,350,[305],1
...,...,...,...,...,...,...
11596,183,"[305, 172, 54, 314]",4,314,"[305, 172]",2
11597,183,"[305, 172, 54, 314, 1]",5,1,"[305, 172]",2
11598,183,"[305, 172, 54, 314]",4,314,"[305, 172, 54]",3
11599,183,"[305, 172, 54, 314, 1]",5,1,"[305, 172, 54]",3


In [80]:
java_new.describe()

,user_id,seq_len,target,position
count,6106.000000,6106.000000,6106.000000,6106.000000
mean,78.041107,16.430396,284.260400,4.984605
std,42.143732,12.348482,88.230569,5.498456
min,1.000000,1.000000,1.000000,0.000000
25%,62.000000,7.000000,300.000000,1.000000
50%,88.000000,13.000000,314.000000,3.000000
75%,95.000000,22.000000,323.000000,7.000000
max,183.000000,66.000000,352.000000,32.000000


In [ ]:
java_new['item_id_list'] = java_new['item_id_list'].apply(lambda x: ''.join(x).strip('[').strip[']'])
java_new

In [86]:
java_new.to_csv('./java/java_train_1.txt',sep='\t',index=False,header=True)

In [72]:
java_user_count = java_new.groupby('user_id').count().reset_index()[['user_id','item_id_list']]
java_user_count

,user_id,item_id_list
0,1,272
1,2,1
2,3,30
3,4,1
4,5,2
...,...,...
174,178,4
175,179,16
176,181,1
177,182,2


In [83]:
java_inter

,user_id,item_id,inter_len
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",33
1,2,"[363, 1]",2
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",11
3,4,"[363, 1]",2
4,5,"[1, 294, 363]",3
...,...,...,...
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]",8
179,180,[1],1
180,181,"[363, 1]",2
181,182,"[294, 54, 1]",3


In [93]:
# ['user_id','item_id_list','seq_len','target','inter_list','position']
java_test_inter = java_inter[java_inter['inter_len']>3]
java_test_inter['item_id_list']=java_test_inter['item_id'].apply(lambda x: x.tolist())
java_test_inter['seq_len']=java_test_inter['inter_len']
java_test_inter['target'] = java_test_inter['item_id'].apply(lambda x: x[-1])
java_test_inter['inter_list'] = java_test_inter['item_id'].apply(lambda x: x[:-1].tolist())
java_test_inter['position']=java_test_inter['inter_len']-1
java_test_inter

/tmp/ipykernel_2767687/1059319601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  java_test_inter['item_id_list']=java_test_inter['item_id'].apply(lambda x: x.tolist())
/tmp/ipykernel_2767687/1059319601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  java_test_inter['seq_len']=java_test_inter['inter_len']
/tmp/ipykernel_2767687/1059319601.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,user_id,item_id,inter_len,item_id_list,seq_len,target,inter_list,position
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",33,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",33,322,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",32
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",11,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",11,35,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 295]",10
5,6,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",33,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",33,203,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",32
7,8,"[294, 295, 35, 297, 298, 54, 300, 301, 302, 30...",17,"[294, 295, 35, 297, 298, 54, 300, 301, 302, 30...",17,321,"[294, 295, 35, 297, 298, 54, 300, 301, 302, 30...",16
8,9,"[327, 321, 294, 301, 300, 310, 173, 336, 350, ...",12,"[327, 321, 294, 301, 300, 310, 173, 336, 350, ...",12,363,"[327, 321, 294, 301, 300, 310, 173, 336, 350, ...",11
...,...,...,...,...,...,...,...,...
164,165,"[1, 294, 295, 35, 297, 298, 300]",7,"[1, 294, 295, 35, 297, 298, 300]",7,300,"[1, 294, 295, 35, 297, 298]",6
175,176,"[363, 1, 294, 300, 310, 321, 322, 173, 342, 361]",10,"[363, 1, 294, 300, 310, 321, 322, 173, 342, 361]",10,361,"[363, 1, 294, 300, 310, 321, 322, 173, 342]",9
177,178,"[298, 311, 1, 331]",4,"[298, 311, 1, 331]",4,331,"[298, 311, 1]",3
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]",8,"[294, 295, 35, 297, 298, 300, 1, 301]",8,301,"[294, 295, 35, 297, 298, 300, 1]",7


In [95]:
java_test_inter[['user_id','item_id_list','seq_len','target','inter_list','position']].to_csv('./java/java_test_1.txt',sep='\t',index=False,header=True)

In [96]:
java_test_inter = java_inter[java_inter['inter_len']>3]
java_test_inter['item_id_list']=java_test_inter['item_id'].apply(lambda x: x.tolist())
java_test_inter['seq_len']=java_test_inter['inter_len']
java_test_inter['target'] = java_test_inter['item_id'].apply(lambda x: x[-1])
java_test_inter['inter_list'] = java_test_inter['item_id'].apply(lambda x: x[:-2].tolist())
java_test_inter['position']=java_test_inter['inter_len']-2
java_test_inter

/tmp/ipykernel_2767687/482758724.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  java_test_inter['item_id_list']=java_test_inter['item_id'].apply(lambda x: x.tolist())
/tmp/ipykernel_2767687/482758724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  java_test_inter['seq_len']=java_test_inter['inter_len']
/tmp/ipykernel_2767687/482758724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

,user_id,item_id,inter_len,item_id_list,seq_len,target,inter_list,position
0,1,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",33,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",33,322,"[305, 172, 54, 314, 1, 350, 352, 301, 302, 310...",31
2,3,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",11,"[363, 294, 173, 297, 322, 301, 311, 310, 1, 29...",11,35,"[363, 294, 173, 297, 322, 301, 311, 310, 1]",9
5,6,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",33,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",33,203,"[1, 363, 297, 300, 310, 314, 312, 301, 302, 31...",31
7,8,"[294, 295, 35, 297, 298, 54, 300, 301, 302, 30...",17,"[294, 295, 35, 297, 298, 54, 300, 301, 302, 30...",17,321,"[294, 295, 35, 297, 298, 54, 300, 301, 302, 30...",15
8,9,"[327, 321, 294, 301, 300, 310, 173, 336, 350, ...",12,"[327, 321, 294, 301, 300, 310, 173, 336, 350, ...",12,363,"[327, 321, 294, 301, 300, 310, 173, 336, 350, 1]",10
...,...,...,...,...,...,...,...,...
164,165,"[1, 294, 295, 35, 297, 298, 300]",7,"[1, 294, 295, 35, 297, 298, 300]",7,300,"[1, 294, 295, 35, 297]",5
175,176,"[363, 1, 294, 300, 310, 321, 322, 173, 342, 361]",10,"[363, 1, 294, 300, 310, 321, 322, 173, 342, 361]",10,361,"[363, 1, 294, 300, 310, 321, 322, 173]",8
177,178,"[298, 311, 1, 331]",4,"[298, 311, 1, 331]",4,331,"[298, 311]",2
178,179,"[294, 295, 35, 297, 298, 300, 1, 301]",8,"[294, 295, 35, 297, 298, 300, 1, 301]",8,301,"[294, 295, 35, 297, 298, 300]",6


In [ ]:
java_test_inter[['user_id','item_id_list','seq_len','target','inter_list','position']].to_csv('./java/java_test_2.txt',sep='\t',index=False,header=True)